<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Predict-expression-modulation-targets" data-toc-modified-id="Predict-expression-modulation-targets-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Predict expression modulation targets</a></span><ul class="toc-item"><li><span><a href="#Succinate-production" data-toc-modified-id="Succinate-production-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Succinate production</a></span></li><li><span><a href="#Flux-Scanning-based-on-Enforced-Objective-Flux" data-toc-modified-id="Flux-Scanning-based-on-Enforced-Objective-Flux-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Flux Scanning based on Enforced Objective Flux</a></span></li><li><span><a href="#Differential-flux-variability-analysis" data-toc-modified-id="Differential-flux-variability-analysis-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Differential flux variability analysis</a></span></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>References</a></span></li></ul></div>

# Predict expression modulation targets

Cameo provides algorithms to search for genes or reactions that can be over or down regulated in order to achieve a given biological objective.

In [ ]:
from cobra.io import read_sbml_model
from cameo import phenotypic_phase_plane

Load the E. coli core model.

In [ ]:
model = read_sbml_model('data/e_coli_core.xml.gz')

## Succinate production

The production envelope looks like this.

In [ ]:
production_envelope = phenotypic_phase_plane(model, 
                                             variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                             objective=model.metabolites.succ_e)
production_envelope.plot()

## Flux Scanning based on Enforced Objective Flux

We wish to find gene amplification targets by searching for reaction fluxes that increase as the production target flux is increased. [[Choi 2010](#references)]
![FSEOF](FSEOF.png)

In [ ]:
from cameo.strain_design.deterministic.flux_variability_based import FSEOF

In [ ]:
fseof = FSEOF(model)

In [ ]:
result = fseof.run(target=model.reactions.EX_succ_e)

In [ ]:
result.data_frame

In [ ]:
from matplotlib import pyplot as plt
result.data_frame.T.plot()
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

## Differential flux variability analysis

Compares flux ranges of a reference model to a set of models that have been parameterized to lie on a grid of evenly spaced points in the n-dimensional production envelope (n being the number of reaction bounds to be varied).

In [ ]:
from cameo.strain_design.deterministic import DifferentialFVA

Set up a model that represents a reference state (in this case a model with a constrained growth rate).

In [ ]:
model.reactions.EX_o2_e.lower_bound = 0
reference_model = model.copy()
biomass_rxn = reference_model.reactions.BIOMASS_Ecoli_core_w_GAM
biomass_rxn.lower_bound = 0.
target = reference_model.metabolites.succ_e

Set up the differential flux variability analysis strain design method.

In [ ]:
diff_fva = DifferentialFVA(design_space_model=model,
                           reference_model=reference_model,
                           objective=target,
                           variables=[biomass_rxn],
                           normalize_ranges_by=biomass_rxn,
                           points=10)

Run differential flux variability analysis (only on the surface of the production envelope)

In [ ]:
%time result = diff_fva.run(surface_only=True)
result.plot()

In [ ]:
result.plot(5, variables=['FBP', 'G6PDH2r', 'PGL', 'PGK'])

In [ ]:
result

In [ ]:
result.display_on_map(5, map_name="iJO1366.Central metabolism")

<a id="references"></a>
# References

[Choi2010]: Choi, H. S., Lee, S. Y., Kim, T. Y., & Woo, H. M. (2010). In silico identification of gene amplification targets for improvement of lycopene production. Applied and Environmental Microbiology, 76(10), 3097–105. https://doi.org/10.1128/AEM.00115-10
